### Dual contouring

https://github.com/BorisTheBrave/mc-dc

In [ ]:
import numpy
import numpy.linalg

from utils_2d import V2
from utils_3d import V3
import settings


            

class QEF:
    """Represents and solves the quadratic error function"""
    def __init__(self, A, b, fixed_values):
        
        self.A = A
        self.b = b
        self.fixed_values = fixed_values
        
    def evaluate(self, x):
        """Evaluates the function at a given point.
        This is what the solve method is trying to minimize.
        NB: Doesn't work with fixed axes."""
        x = numpy.array(x)
        return numpy.linalg.norm(numpy.matmul(self.A, x) - self.b)

    def eval_with_pos(self, x):
        """Evaluates the QEF at a position, returning the same format solve does."""
        return self.evaluate(x), x

    @staticmethod
    def make_2d(positions, normals):
        """Returns a QEF that measures the the error from a bunch of normals, each emanating from given positions"""
        A = numpy.array(normals)
        b = [v[0] * n[0] + v[1] * n[1] for v, n in zip(positions, normals)]
        fixed_values = [None] * A.shape[1]
        return QEF(A, b, fixed_values)

    @staticmethod
    def make_3d(positions, normals):
        """Returns a QEF that measures the the error from a bunch of normals, each emanating from given positions"""
        A = numpy.array(normals)
        b = [v[0] * n[0] + v[1] * n[1] + v[2] * n[2] for v, n in zip(positions, normals)]
        fixed_values = [None] * A.shape[1]
        # Ensure the same lenght
        
        
#         if len(b) > len(A):
#             l = len(A)
#             while len(b) > l:
#                 b.pop(-1)    
#         if len(A) > len(b):
#             l = len(b)
# #             print(f"A:{len(A)}, B:{len(b)} Needed to remove {abs(len(A)-len(b))} items...",end="")
#             new_a = []
#             for index, i in enumerate(A):
#                 if index == l:
#                     break
#                 new_a.append(i)
#             A = new_a[:]
# #             print(f"Done now is A:{len(A)}==b:{len(b)}")
            
            
        return QEF(A, b, fixed_values)

    def fix_axis(self, axis, value):
        """Returns a new QEF that gives the same values as the old one, only with the position along the given axis
        constrained to be value."""
        # Pre-evaluate the fixed axis, adjusting b
        b = self.b[:] - self.A[:, axis] * value
        # Remove that axis from a
        A = numpy.delete(self.A, axis, 1)
        fixed_values = self.fixed_values[:]
        fixed_values[axis] = value
        return QEF(A, b, fixed_values)

    def solve(self):
        """Finds the point that minimizes the error of this QEF,
        and returns a tuple of the error squared and the point itself"""

        result, residual, rank, s = numpy.linalg.lstsq(self.A, self.b)
        if len(residual) == 0:
            residual = self.evaluate(result)
        else:
            residual = residual[0]
        # Result only contains the solution for the unfixed axis,
        # we need to add back all the ones we previously fixed.
        position = []
        i = 0
        for value in self.fixed_values:
            if value is None:
                position.append(result[i])
                i += 1
            else:
                position.append(value)
        return residual, position


def solve_qef_2d(x, y, positions, normals):
    # The error term we are trying to minimize is sum( dot(x-v[i], n[i]) ^ 2)
    # This should be minimized over the unit square with top left point (x, y)

    # In other words, minimize || A * x - b || ^2 where A and b are a matrix and vector
    # derived from v and n
    # The heavy lifting is done by the QEF class, but this function includes some important
    # tricks to cope with edge cases

    # This is demonstration code and isn't optimized, there are many good C++ implementations
    # out there if you need speed.

    if settings.BIAS:
        # Add extra normals that add extra error the further we go
        # from the cell, this encourages the final result to be
        # inside the cell
        # These normals are shorter than the input normals
        # as that makes the bias weaker,  we want them to only
        # really be important when the input is ambiguous

        # Take a simple average of positions as the point we will
        # pull towards.
        mass_point = numpy.mean(positions, axis=0)

        normals.append([settings.BIAS_STRENGTH, 0])
        positions.append(mass_point)
        normals.append([0, settings.BIAS_STRENGTH])
        positions.append(mass_point)

    qef = QEF.make_2d(positions, normals)

    residual, v = qef.solve()

    if settings.BOUNDARY:
        def inside(r):
            return x <= r[1][0] <= x + 1 and y <= r[1][1] <= y + 1

        # It's entirely possible that the best solution to the qef is not actually
        # inside the cell.
        if not inside((residual, v)):
            # If so, we constrain the the qef to the horizontal and vertical
            # lines bordering the cell, and find the best point of those
            r1 = qef.fix_axis(0, x + 0).solve()
            r2 = qef.fix_axis(0, x + 1).solve()
            r3 = qef.fix_axis(1, y + 0).solve()
            r4 = qef.fix_axis(1, y + 1).solve()

            rs = list(filter(inside, [r1, r2, r3, r4]))

            if len(rs) == 0:
                # It's still possible that those lines (which are infinite)
                # cause solutions outside the box. So finally, we evaluate which corner
                # of the cell looks best
                r1 = qef.eval_with_pos((x + 0, y + 0))
                r2 = qef.eval_with_pos((x + 0, y + 1))
                r3 = qef.eval_with_pos((x + 1, y + 0))
                r4 = qef.eval_with_pos((x + 1, y + 1))

                rs = list(filter(inside, [r1, r2, r3, r4]))

            # Pick the best of the available options
            residual, v = min(rs)

    if settings.CLIP:
        # Crudely force v to be inside the cell
        v[0] = numpy.clip(v[0], x, x + 1)
        v[1] = numpy.clip(v[1], y, y + 1)

    return V2(v[0], v[1])


def solve_qef_3d(x, y, z, positions, normals):
    # The error term we are trying to minimize is sum( dot(x-v[i], n[i]) ^ 2)
    # This should be minimized over the unit square with top left point (x, y)

    # In other words, minimize || A * x - b || ^2 where A and b are a matrix and vector
    # derived from v and n
    # The heavy lifting is done by the QEF class, but this function includes some important
    # tricks to cope with edge cases

    # This is demonstration code and isn't optimized, there are many good C++ implementations
    # out there if you need speed.

    if settings.BIAS:
        # Add extra normals that add extra error the further we go
        # from the cell, this encourages the final result to be
        # inside the cell
        # These normals are shorter than the input normals
        # as that makes the bias weaker,  we want them to only
        # really be important when the input is ambiguous

        # Take a simple average of positions as the point we will
        # pull towards.
        mass_point = numpy.mean(positions, axis=0)

        normals.append([settings.BIAS_STRENGTH, 0, 0])
        positions.append(mass_point)
        normals.append([0, settings.BIAS_STRENGTH, 0])
        positions.append(mass_point)
        normals.append([0, 0, settings.BIAS_STRENGTH])
        positions.append(mass_point)

    qef = QEF.make_3d(positions, normals)

    residual, v = qef.solve()

    if settings.BOUNDARY:
        def inside(r):
            return x <= r[1][0] <= x + 1 and y <= r[1][1] <= y + 1 and z <= r[1][2] <= z + 1

        # It's entirely possible that the best solution to the qef is not actually
        # inside the cell.
        if not inside((residual, v)):
            # If so, we constrain the the qef to the 6
            # planes bordering the cell, and find the best point of those
            r1 = qef.fix_axis(0, x + 0).solve()
            r2 = qef.fix_axis(0, x + 1).solve()
            r3 = qef.fix_axis(1, y + 0).solve()
            r4 = qef.fix_axis(1, y + 1).solve()
            r5 = qef.fix_axis(2, z + 0).solve()
            r6 = qef.fix_axis(2, z + 1).solve()

            rs = list(filter(inside, [r1, r2, r3, r4, r5, r6]))

            if len(rs) == 0:
                # It's still possible that those planes (which are infinite)
                # cause solutions outside the box.
                # So now try the 12 lines bordering the cell
                r1  = qef.fix_axis(1, y + 0).fix_axis(0, x + 0).solve()
                r2  = qef.fix_axis(1, y + 1).fix_axis(0, x + 0).solve()
                r3  = qef.fix_axis(1, y + 0).fix_axis(0, x + 1).solve()
                r4  = qef.fix_axis(1, y + 1).fix_axis(0, x + 1).solve()
                r5  = qef.fix_axis(2, z + 0).fix_axis(0, x + 0).solve()
                r6  = qef.fix_axis(2, z + 1).fix_axis(0, x + 0).solve()
                r7  = qef.fix_axis(2, z + 0).fix_axis(0, x + 1).solve()
                r8  = qef.fix_axis(2, z + 1).fix_axis(0, x + 1).solve()
                r9  = qef.fix_axis(2, z + 0).fix_axis(1, y + 0).solve()
                r10 = qef.fix_axis(2, z + 1).fix_axis(1, y + 0).solve()
                r11 = qef.fix_axis(2, z + 0).fix_axis(1, y + 1).solve()
                r12 = qef.fix_axis(2, z + 1).fix_axis(1, y + 1).solve()

                rs = list(filter(inside, [r1, r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12]))

            if len(rs) == 0:
                # So finally, we evaluate which corner
                # of the cell looks best
                r1 = qef.eval_with_pos((x + 0, y + 0, z + 0))
                r2 = qef.eval_with_pos((x + 0, y + 0, z + 1))
                r3 = qef.eval_with_pos((x + 0, y + 1, z + 0))
                r4 = qef.eval_with_pos((x + 0, y + 1, z + 1))
                r5 = qef.eval_with_pos((x + 1, y + 0, z + 0))
                r6 = qef.eval_with_pos((x + 1, y + 0, z + 1))
                r7 = qef.eval_with_pos((x + 1, y + 1, z + 0))
                r8 = qef.eval_with_pos((x + 1, y + 1, z + 1))

                rs = list(filter(inside, [r1, r2, r3, r4, r5, r6, r7, r8]))

            # Pick the best of the available options
            residual, v = min(rs)

    if settings.CLIP:
        # Crudely force v to be inside the cell
        v[0] = numpy.clip(v[0], x, x + 1)
        v[1] = numpy.clip(v[1], y, y + 1)
        v[2] = numpy.clip(v[2], z, z + 1)

    return V3(v[0], v[1], v[2])

In [ ]:
"""Provides a function for performing 3D Dual Countouring"""
import importlib
from common import adapt
from settings import ADAPTIVE, XMIN, XMAX, YMIN, YMAX, ZMIN, ZMAX
import numpy as np
import math
from utils_3d import V3, Quad, Mesh, make_obj

def dual_contour_3d_find_best_vertex(f, f_normal, x, y, z):
    global d
    if not ADAPTIVE:
        return V3(x+0.5, y+0.5, z+0.5)

    # Evaluate f at each corner
    v = np.empty((2, 2, 2))
    for dx in (0, 1):
        for dy in (0, 1):
            for dz in (0,1):
                v[dx, dy, dz] = f(x + dx, y + dy, z + dz)

    # For each edge, identify where there is a sign change.
    # There are 4 edges along each of the three axes
    changes = []
    for dx in (0, 1):
        for dy in (0, 1):
            if (v[dx, dy, 0] > 0) != (v[dx, dy, 1] > 0):
                changes.append((x + dx, y + dy, z + adapt(v[dx, dy, 0], v[dx, dy, 1])))

    for dx in (0, 1):
        for dz in (0, 1):
            if (v[dx, 0, dz] > 0) != (v[dx, 1, dz] > 0):
                changes.append((x + dx, y + adapt(v[dx, 0, dz], v[dx, 1, dz]), z + dz))

    for dy in (0, 1):
        for dz in (0, 1):
            if (v[0, dy, dz] > 0) != (v[1, dy, dz] > 0):
                changes.append((x + adapt(v[0, dy, dz], v[1, dy, dz]), y + dy, z + dz))

    if len(changes) <= 1:
        return None

    # For each sign change location v[i], we find the normal n[i].
    # The error term we are trying to minimize is sum( dot(x-v[i], n[i]) ^ 2)

    # In other words, minimize || A * x - b || ^2 where A and b are a matrix and vector
    # derived from v and n
    normals = []
    for v in changes:
        n = f_normal(v[0], v[1], v[2])
        normals.append([n.x, n.y, n.z])
        solved = solve_qef_3d(x, y, z, changes, normals)
    return solved


def dual_contour_3d(f, f_normal, xmin=XMIN, ymin=YMIN, zmin=ZMIN, xmax=XMAX, ymax=YMAX, zmax=ZMAX):
    """Iterates over a cells of size one between the specified range, and evaluates f and f_normal to produce
        a boundary by Dual Contouring. Returns a Mesh object."""
    # For each cell, find the the best vertex for fitting f
    vert_array = []
    vert_indices = {}
    print(xmax, ymax,zmax)
    for x in range(xmin, xmax):
        print(x,end=", ")
        for y in range(ymin, ymax):
            for z in range(zmin, zmax):
                try:
                    vert = dual_contour_3d_find_best_vertex(f, f_normal, x, y, z)
                    if vert is None:
                        continue
                    vert_array.append(vert)
                    vert_indices[(x, y, z)] = len(vert_array)
                except:
                    pass

    # For each cell edge, emit an face between the center of the adjacent cells if it is a sign changing edge
    faces = []
    for x in range(xmin, xmax):
        for y in range(ymin, ymax):
            for z in range(ymin, ymax):
                try:
                    if x > xmin and y > ymin:
                        solid1 = f(x, y, z + 0) > 0
                        solid2 = f(x, y, z + 1) > 0
                        if solid1 != solid2:
                            faces.append(Quad(
                                vert_indices[(x - 1, y - 1, z)],
                                vert_indices[(x - 0, y - 1, z)],
                                vert_indices[(x - 0, y - 0, z)],
                                vert_indices[(x - 1, y - 0, z)],
                            ).swap(solid2))
                    if x > xmin and z > zmin:
                        solid1 = f(x, y + 0, z) > 0
                        solid2 = f(x, y + 1, z) > 0
                        if solid1 != solid2:
                            faces.append(Quad(
                                vert_indices[(x - 1, y, z - 1)],
                                vert_indices[(x - 0, y, z - 1)],
                                vert_indices[(x - 0, y, z - 0)],
                                vert_indices[(x - 1, y, z - 0)],
                            ).swap(solid1))
                    if y > ymin and z > zmin:
                        solid1 = f(x + 0, y, z) > 0
                        solid2 = f(x + 1, y, z) > 0
                        if solid1 != solid2:
                            try:
                                faces.append(Quad(
                                vert_indices[(x, y - 1, z - 1)],
                                vert_indices[(x, y - 0, z - 1)],
                                vert_indices[(x, y - 0, z - 0)],
                                vert_indices[(x, y - 1, z - 0)],
                                ).swap(solid2))
                            except:
                                pass
                except Exception as e:
                    print(f"This happend: {e}")
                    pass

    return Mesh(vert_array, faces)


def circle_function(x, y, z):
    return 2.5 - math.sqrt(x*x + y*y + z*z)


def circle_normal(x, y, z):
    l = math.sqrt(x*x + y*y + z*z)
    return V3(-x / l, -y / l, -z / l)

def intersect_function(x, y, z):
    y -= 0.3
    x -= 0.5
    x = abs(x)
    return min(x - y, x + y)

def normal_from_function(f, d=0.01):
    """Given a sufficiently smooth 3d function, f, returns a function approximating of the gradient of f.
    d controls the scale, smaller values are a more accurate approximation."""
    def norm(x, y, z):
        x2 =  (f(x + d, y, z) - f(x - d, y, z)+0.0001) / 2 / d
        y2 = (f(x, y + d, z) - f(x, y - d, z)+0.00001) / 2 / d
        z2 = (f(x, y, z + d) - f(x, y, z - d)+0.00001) / 2 / d
        v3 = V3(x2,y2,z2)
        return v3.normalize()
    return norm

def f(x,y,z):
    try:
        if label_data[round(x)][round(y)][round(z)] > isovalue:
            return 1
        return -1
    except:
        return -1
f_normal =  normal_from_function(f)

# Make sure numpy is installed with conda or this does not work on entry to dlascl parameter number 4 had an illegal value

In [ ]:
size = map(lambda x: x-1, label_data.shape)
mesh = dual_contour_3d(f, f_normal,1,1,1,*size)
print(mesh)
print(mesh.faces)
# did not get this to work after 6 hours :(

In [ ]:
pymesh.save_mesh_raw("bone.obj", np.array(mesh.verts), np.array(mesh.faces))

In [ ]:
pymesh.save_mesh_raw("bonedualcontour .obj", np.array(mesh.verts), np.array(mesh.faces))